In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import scikeras
import pandas as pd

import os
import pickle

E:\Anaconda3\envs\ML_new\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
E:\Anaconda3\envs\ML_new\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
E:\Anaconda3\envs\ML_new\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# method used for saving object as pickle
def save_object_as_pickle(obj, filename):
    with open(filename, 'wb') as file:
        pickle.dump(obj, file, pickle.HIGHEST_PROTOCOL)

In [3]:
# LOAD THE DATA 
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.boston_housing.load_data()

### Tensorboard

In [4]:
root_logdir = os.path.join(os.curdir, "tb_logs")
def get_run_logdir(parameter, value):
    import time
    run_id = str(int(time.time())) + "_" + parameter + "_" + str(value)
    return os.path.join(root_logdir, run_id)

#### Callbacks

In [5]:
def get_callbacks_list(parameter, value):
    checkpoint_cb = keras.callbacks.ModelCheckpoint(parameter + "_" +  str(value)  + ".h5")
    run_logdir = get_run_logdir(parameter, value)
    tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

    early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, min_delta=1.0)

    callbacks_list=[
        # checkpoint_cb,
        tensorboard_cb,
        early_stopping_cb]
    return callbacks_list

### Create build_model method

In [6]:
def build_model(n_hidden=1, n_neurons=25, learning_rate=10e-5, input_shape=[13], optimizer="sgd", momentum=0):
    model = keras.models.Sequential() 
    model.add(keras.layers.InputLayer(input_shape=input_shape)) 
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    if optimizer == "sgd":
        optimizer_prep = keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "nesterov":
        optimizer_prep = keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True, momentum=momentum)
    elif optimizer == "momentum":
        optimizer_prep = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)
    elif optimizer == "adam":
        optimizer_prep = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss=['mse', 'mae'], optimizer=optimizer_prep, metrics=['mse', 'mae'])
    return model

### Learning rate test 

In [7]:
lr_test = [10**(-6),  10**(-5), 10**(-4)]
lr_values = []
lr_mse = []
lr_mae = []

In [8]:
# clear TensorFlow session
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# perform tests
for lr_val in lr_test:
    callbacks_list = get_callbacks_list("lr", lr_val)
    model = build_model(input_shape=13, learning_rate=lr_val)
    history = model.fit(
        X_train, 
        y_train, 
        epochs=100, 
        validation_split=0.1,
        callbacks=callbacks_list,
        verbose=False
    )
    lr_values.append(lr_val)
    print(lr_val)
    mse_val = history.history['mse'][-1] 
    lr_mse.append(mse_val)
    print(mse_val)
    mae_val = history.history['mae'][-1] 
    lr_mae.append(mae_val)
    print(mae_val)
    print("-------------")
    # y_pred = model.predict(X_test)
    # mse_test = tf.keras.metrics.mean_squared_error(y_test, y_pred).numpy()
    # mae_test = tf.keras.metrics.mean_absolute_error(y_test, y_pred).numpy()
    # print("mse_test:", np.mean(mse_test))
    # print("mse_test:", np.mean(mae_test))

1e-06
89.73018646240234
6.852303504943848
-------------
1e-05
75.02506256103516
6.1999664306640625
-------------
0.0001
182.50698852539062
10.223281860351562
-------------


### Tensorboard analysis

In [9]:
# %load_ext tensorboard
# %tensorboard --logdir ./my_logs --port=6006

### Hidden layers test 

In [10]:
hl_test = [0, 1, 2, 3]
hl_values = []
hl_mse = []
hl_mae = []


In [11]:
# clear TensorFlow session
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# perform tests
for hl_val in hl_test:
    callbacks_list = get_callbacks_list("hl", hl_val)
    model = build_model(input_shape=13, n_hidden=hl_val)
    history = model.fit(
        X_train, 
        y_train, 
        epochs=100, 
        validation_split=0.1,
        callbacks=callbacks_list,
        verbose=False
    )
    hl_values.append(hl_val)
    print(hl_val)
    mse_val = history.history['mse'][-1] 
    hl_mse.append(mse_val)
    print(mse_val)
    mae_val = history.history['mae'][-1]
    hl_mae.append(mae_val)
    print(mae_val)
    print("-------------")
    # y_pred = model.predict(X_test)
    # mse_test = tf.keras.metrics.mean_squared_error(y_test, y_pred).numpy()
    # mae_test = tf.keras.metrics.mean_absolute_error(y_test, y_pred).numpy()
    # print("mse_test:", np.mean(mse_test))
    # print("mse_test:", np.mean(mae_test))

0
nan
nan
-------------
1
423.1585693359375
18.273088455200195
-------------
2
89.60796356201172
6.7229156494140625
-------------
3
nan
nan
-------------


### Tensorboard analysis

In [12]:
# %load_ext tensorboard
# %tensorboard --logdir ./my_logs --port=6006

### N_neurons test 

In [13]:
nn_test = [5, 25, 125]
nn_values = []
nn_mse = []
nn_mae = []


In [14]:
# clear TensorFlow session
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# perform tests
for nn_val in nn_test:
    callbacks_list = get_callbacks_list("nn", nn_val)
    model = build_model(input_shape=13, n_neurons=nn_val)
    history = model.fit(
        X_train, 
        y_train,
        validation_split=0.1,
        epochs=100, 
        callbacks=callbacks_list,
        verbose=False
    )
    nn_values.append(nn_val)
    print(nn_val)
    mse_val = history.history['mse'][-1] 
    nn_mse.append(mse_val)
    print(mse_val)
    mae_val = history.history['mae'][-1] 
    nn_mae.append(mae_val)
    print(mae_val)
    print("-------------")
    # y_pred = model.predict(X_test)
    # mse_test = tf.keras.metrics.mean_squared_error(y_test, y_pred).numpy()
    # mae_test = tf.keras.metrics.mean_absolute_error(y_test, y_pred).numpy()
    # print("mse_test:", np.mean(mse_test))
    # print("mse_test:", np.mean(mae_test))

5
419.0284423828125
18.15972328186035
-------------
25
408.7922668457031
17.875659942626953
-------------
125
377.2396240234375
16.975788116455078
-------------


### Tensorboard analysis

In [15]:
# %load_ext tensorboard
# %tensorboard --logdir ./my_logs --port=6006

### Optimization algorithm test 

In [16]:
opt_test = ["sgd", "nesterov", "momentum", "adam"]
opt_values = []
opt_mse = []
opt_mae = []

In [17]:
# clear TensorFlow session
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# perform tests
for opt_val in opt_test:
    callbacks_list = get_callbacks_list("opt", opt_val)
    model = build_model(input_shape=13, optimizer=opt_val)
    history = model.fit(
        X_train, 
        y_train, 
        epochs=100, 
        validation_split=0.1,
        callbacks=callbacks_list,
        verbose=False
    )
    opt_values.append(opt_val)
    print(opt_val)
    mse_val = history.history['mse'][-1]
    opt_mse.append(mse_val)
    print(mse_val)
    mae_val = history.history['mae'][-1]
    opt_mae.append(mae_val)
    print(mae_val)
    print("-------------")
    # y_pred = model.predict(X_test)
    # mse_test = tf.keras.metrics.mean_squared_error(y_test, y_pred).numpy()
    # mae_test = tf.keras.metrics.mean_absolute_error(y_test, y_pred).numpy()
    # print("mse_test:", np.mean(mse_test))
    # print("mse_test:", np.mean(mae_test))

sgd
220.78456115722656
11.750006675720215
-------------
nesterov
408.7922668457031
17.875659942626953
-------------
momentum
182.50698852539062
10.223281860351562
-------------
adam
318.7983093261719
12.836956977844238
-------------


### Tensorboard analysis

In [18]:
# %load_ext tensorboard
# %tensorboard --logdir ./my_logs --port=6006

### Momentum test 

In [19]:
mom_test = [0.1, 0.5, 0.9]
mom_values = []
mom_mse = []
mom_mae = []

In [20]:
# clear TensorFlow session
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# perform tests
for mom_val in mom_test:
    callbacks_list = get_callbacks_list("mom", mom_val)
    model = build_model(input_shape=13, optimizer='momentum', momentum=mom_val)
    history = model.fit(
        X_train, 
        y_train, 
        epochs=100,
        validation_split=0.1,
        callbacks=callbacks_list,
        verbose=False
    )
    mom_values.append(mom_val)
    print(mom_val)
    mse_val = history.history['mse'][-1] 
    mom_mse.append(mse_val)
    print(mse_val)
    mae_val = history.history['mae'][-1]
    mom_mae.append(mae_val)
    print(mae_val)
    print("-------------")
    # y_pred = model.predict(X_test)
    # mse_test = tf.keras.metrics.mean_squared_error(y_test, y_pred).numpy()
    # mae_test = tf.keras.metrics.mean_absolute_error(y_test, y_pred).numpy()
    # print("mse_test:", np.mean(mse_test))
    # print("mse_test:", np.mean(mae_test))

0.1
199.33689880371094
10.909239768981934
-------------
0.5
291.0889892578125
14.264500617980957
-------------
0.9
4295.2060546875
64.85430908203125
-------------


### Tensorboard analysis

In [21]:
# %load_ext tensorboard
# %tensorboard --logdir ./tb_logs --port=6006

## Save values in pickle

In [22]:
# learning rate
lr_data = []
for i in range(len(lr_test)):
    lr_data.append((lr_values[i], lr_mse[i], lr_mae[i]))

print(lr_data)
save_object_as_pickle(lr_data, "lr.pkl")

[(1e-06, 89.73018646240234, 6.852303504943848), (1e-05, 75.02506256103516, 6.1999664306640625), (0.0001, 182.50698852539062, 10.223281860351562)]


In [23]:
# hidden layers 
hl_data = []
for i in range(len(hl_test)):
    hl_data.append((hl_values[i], hl_mse[i], hl_mae[i]))

print(hl_data)
save_object_as_pickle(hl_data, "hl.pkl")

[(0, nan, nan), (1, 423.1585693359375, 18.273088455200195), (2, 89.60796356201172, 6.7229156494140625), (3, nan, nan)]


In [24]:
# n_neurons
nn_data = []
for i in range(len(nn_test)):
    nn_data.append((nn_values[i], nn_mse[i], nn_mae[i]))

print(nn_data)
save_object_as_pickle(nn_data, "nn.pkl")

[(5, 419.0284423828125, 18.15972328186035), (25, 408.7922668457031, 17.875659942626953), (125, 377.2396240234375, 16.975788116455078)]


In [25]:
# optimization
opt_data = []
for i in range(len(opt_test)):
    opt_data.append((opt_values[i], opt_mse[i], opt_mae[i]))

print(opt_data)
save_object_as_pickle(opt_data, "opt.pkl")

[('sgd', 220.78456115722656, 11.750006675720215), ('nesterov', 408.7922668457031, 17.875659942626953), ('momentum', 182.50698852539062, 10.223281860351562), ('adam', 318.7983093261719, 12.836956977844238)]


In [26]:
# momentum
mom_data = []
for i in range(len(mom_test)):
    mom_data.append((mom_values[i], mom_mse[i], mom_mae[i]))

print(mom_data)
save_object_as_pickle(mom_data, "mom.pkl")

[(0.1, 199.33689880371094, 10.909239768981934), (0.5, 291.0889892578125, 14.264500617980957), (0.9, 4295.2060546875, 64.85430908203125)]


### TASK 2.2

In [27]:
# TODO  

In [28]:
param_distribs = {
    "model__n_hidden": [0, 1, 2, 3],
    "model__n_neurons": [5, 25, 125],
    "model__learning_rate": [10**(-6),  10**(-5), 10**(-4)],
    "model__optimizer": ["sgd", "nesterov", "momentum", "adam"],
    "model__momentum": [0.1, 0.5, 0.9]
}


In [29]:
import scikeras
from scikeras.wrappers import KerasRegressor
es = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=1.0, verbose=1)
keras_reg = KerasRegressor(build_model, callbacks=[es])


In [30]:
from sklearn.model_selection import RandomizedSearchCV
rnd_search_cv = RandomizedSearchCV(keras_reg,
                                   param_distribs,
                                   n_iter=30,
                                   cv=3,
                                   verbose=2
                                  )
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_split=0.1)


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Epoch 1/100
8/8 [==============================] - 0s 20ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 2/100
8/8 [==============================] - 0s 4ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 3/100
8/8 [==============================] - 0s 4ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 4/100
8/8 [==============================] - 0s 5ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 5/100
8/8 [==============================] - 0s 4ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 6/100
8/8 [==============================] - 0s 4ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 7/100
8/8 [==============================] - 0s 4ms/step - loss: nan - mse

E:\Anaconda3\envs\ML_new\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\scikeras\wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\scikeras\wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\sklearn\metrics\_regression.py", line 789, in r2_score
    y_type, y_tru

8/8 [==============================] - 0s 18ms/step - loss: 7657043506052988928.0000 - mse: 7657043506052988928.0000 - mae: 1007819584.0000 - val_loss: 2308499570688.0000 - val_mse: 2308499570688.0000 - val_mae: 1519374.6250
Epoch 2/100
8/8 [==============================] - 0s 4ms/step - loss: 2305448214528.0000 - mse: 2305448214528.0000 - mae: 1518370.1250 - val_loss: 2301122838528.0000 - val_mse: 2301122838528.0000 - val_mae: 1516945.1250
Epoch 3/100
8/8 [==============================] - 0s 4ms/step - loss: 2298081443840.0000 - mse: 2298081443840.0000 - mae: 1515942.2500 - val_loss: 2293769961472.0000 - val_mse: 2293769961472.0000 - val_mae: 1514519.7500
Epoch 4/100
8/8 [==============================] - 0s 4ms/step - loss: 2290738528256.0000 - mse: 2290738528256.0000 - mae: 1513518.5000 - val_loss: 2286441201664.0000 - val_mse: 2286441201664.0000 - val_mae: 1512098.2500
Epoch 5/100
8/8 [==============================] - 0s 4ms/step - loss: 2283419205632.0000 - mse: 2283419205632.0

E:\Anaconda3\envs\ML_new\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\scikeras\wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\scikeras\wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\sklearn\metrics\_regression.py", line 789, in r2_score
    y_type, y_tru

8/8 [==============================] - 0s 19ms/step - loss: 21785.2227 - mse: 21785.2227 - mae: 146.3550 - val_loss: 20512.6641 - val_mse: 20512.6641 - val_mae: 142.1184
Epoch 2/100
8/8 [==============================] - 0s 4ms/step - loss: 21782.3613 - mse: 21782.3613 - mae: 146.3454 - val_loss: 20509.8711 - val_mse: 20509.8711 - val_mae: 142.1087
Epoch 3/100
8/8 [==============================] - 0s 4ms/step - loss: 21779.5137 - mse: 21779.5137 - mae: 146.3358 - val_loss: 20507.0664 - val_mse: 20507.0664 - val_mae: 142.0990
Epoch 4/100
8/8 [==============================] - 0s 4ms/step - loss: 21776.6660 - mse: 21776.6660 - mae: 146.3262 - val_loss: 20504.2734 - val_mse: 20504.2734 - val_mae: 142.0893
Epoch 5/100
8/8 [==============================] - 0s 4ms/step - loss: 21773.8379 - mse: 21773.8379 - mae: 146.3167 - val_loss: 20501.4668 - val_mse: 20501.4668 - val_mae: 142.0796
Epoch 6/100
8/8 [==============================] - 0s 4ms/step - loss: 21770.9648 - mse: 21770.9648 - mae:

...

E:\Anaconda3\envs\ML_new\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\scikeras\wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\scikeras\wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\sklearn\metrics\_regression.py", line 789, in r2_score
    y_type, y_tru

8/8 [==============================] - 0s 17ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 2/100
8/8 [==============================] - 0s 4ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 3/100
8/8 [==============================] - 0s 4ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 4/100
8/8 [==============================] - 0s 3ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 5/100
8/8 [==============================] - 0s 4ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 6/100
8/8 [==============================] - 0s 4ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 7/100
8/8 [==============================] - 0s 3ms/step - loss: nan - mse: nan - mae: nan - val_loss: nan - val_mse: nan - val_mae: nan
Epoch 8/10

E:\Anaconda3\envs\ML_new\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\sklearn\metrics\_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\scikeras\wrappers.py", line 1100, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\scikeras\wrappers.py", line 1697, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "E:\Anaconda3\envs\ML_new\lib\site-packages\sklearn\metrics\_regression.py", line 789, in r2_score
    y_type, y_tru

12/12 [==============================] - 0s 12ms/step - loss: 1156.2904 - mse: 1156.2904 - mae: 31.1206 - val_loss: 191.0138 - val_mse: 191.0138 - val_mae: 12.3603
Epoch 2/100
12/12 [==============================] - 0s 3ms/step - loss: 157.8184 - mse: 157.8184 - mae: 9.8982 - val_loss: 131.9712 - val_mse: 131.9712 - val_mae: 8.8789
Epoch 3/100
12/12 [==============================] - 0s 3ms/step - loss: 141.0938 - mse: 141.0938 - mae: 9.7339 - val_loss: 50.9592 - val_mse: 50.9592 - val_mae: 6.0747
Epoch 4/100
12/12 [==============================] - 0s 3ms/step - loss: 76.8893 - mse: 76.8893 - mae: 6.3340 - val_loss: 26.7590 - val_mse: 26.7590 - val_mae: 4.0263
Epoch 5/100
12/12 [==============================] - 0s 3ms/step - loss: 75.2245 - mse: 75.2245 - mae: 5.9670 - val_loss: 25.7716 - val_mse: 25.7716 - val_mae: 4.1436
Epoch 6/100
12/12 [==============================] - 0s 3ms/step - loss: 71.2877 - mse: 71.2877 - mae: 6.1335 - val_loss: 29.7247 - val_mse: 29.7247 - val_mae: 4.

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x000001854F5A5B80>], model=<function build_model at 0x00000185426F9CA0>),
                   n_iter=30,
                   param_distributions={'model__learning_rate': [1e-06, 1e-05,
                                                                 0.0001],
                                        'model__momentum': [0.1, 0.5, 0.9],
                                        'model__n_hidden': [0, 1, 2, 3],
                                        'model__n_neurons': [5, 25, 125],
                                        'model__optimizer': ['sgd', 'nesterov',
                                                             'momentum',
                                                             'adam']},
                   verbose=2)

In [31]:
print(rnd_search_cv.best_score_, rnd_search_cv.best_params_)

0.35665628721032966 {'model__optimizer': 'adam', 'model__n_neurons': 125, 'model__n_hidden': 3, 'model__momentum': 0.1, 'model__learning_rate': 0.0001}


In [32]:
save_object_as_pickle(rnd_search_cv.best_params_, "rnd_search.pkl")

### Check saved data

In [33]:
# check if pickles' contents are saved correctly

print("lr.pkl\n", pd.read_pickle("lr.pkl"), "\n")
print("hl.pkl\n", pd.read_pickle("hl.pkl"), "\n")
print("nn.pkl\n", pd.read_pickle("nn.pkl"), "\n")
print("opt.pkl\n", pd.read_pickle("opt.pkl"), "\n")
print("mom.pkl\n", pd.read_pickle("mom.pkl"), "\n")
print("rnd_search.pkl\n", pd.read_pickle("rnd_search.pkl"), "\n")


lr.pkl
 [(1e-06, 89.73018646240234, 6.852303504943848), (1e-05, 75.02506256103516, 6.1999664306640625), (0.0001, 182.50698852539062, 10.223281860351562)] 

hl.pkl
 [(0, nan, nan), (1, 423.1585693359375, 18.273088455200195), (2, 89.60796356201172, 6.7229156494140625), (3, nan, nan)] 

nn.pkl
 [(5, 419.0284423828125, 18.15972328186035), (25, 408.7922668457031, 17.875659942626953), (125, 377.2396240234375, 16.975788116455078)] 

opt.pkl
 [('sgd', 220.78456115722656, 11.750006675720215), ('nesterov', 408.7922668457031, 17.875659942626953), ('momentum', 182.50698852539062, 10.223281860351562), ('adam', 318.7983093261719, 12.836956977844238)] 

mom.pkl
 [(0.1, 199.33689880371094, 10.909239768981934), (0.5, 291.0889892578125, 14.264500617980957), (0.9, 4295.2060546875, 64.85430908203125)] 

rnd_search.pkl
 {'model__optimizer': 'adam', 'model__n_neurons': 125, 'model__n_hidden': 3, 'model__momentum': 0.1, 'model__learning_rate': 0.0001} 

